#  Géo API

## Description of Notebook 

This Notebook imports a CSV file into a dataframe, treats all the positions amd returns a new dataframe with: 
- housenumber
- address
- distance of point to position

Finally, it exports the result to a new CSV file. 


## Import Libraries

In [ ]:
import pandas as pd
import chardet
import time

detect CSV file encoding and return the Encoding. 

In [ ]:
filePath='E:/Feina/Autonoms/CTG_Project/AOAngers3/travailW21/ETL_Angers_20190526.csv'
with open(filePath, 'rb') as f:
    result = chardet.detect(f.read())
result

## Import of the data 

Opens the file with the found encoding. Usually 'UTF-8' (but not always)

the separator is set to ';'

Store the data in a dataframe

In [ ]:
data= pd.read_csv(filePath, sep=';', header='infer', encoding=result['encoding'])

API querying to the "api-adresse.data.gouv.fr" website. 

Store the results in three separate lists. 

In [ ]:
# list creation
housenbr=[]
rue=[]
distance=[]

for i in range(data.shape[0]):
    long, lat =(str(data['X'][i])), (str(data['Y'][i]))
# if decimal separator is a comma, replace it by a . 
    long=long.replace(',','.')
    lat=lat.replace(',','.')

#  print(long, lat)
    url='https://api-adresse.data.gouv.fr/reverse/?lon=%s&lat=%s'%(long, lat)
    print(url)
    r = requests.get(url)
    response=r.json(); res2=response['features']
    try: num= res2[0]['properties']['housenumber']
    except: num = 'vide'
    housenbr.append(num)
    try: stret= res2[0]['properties']['street']
    except: stret= res2[0]['properties']['name']
    rue.append(stret)
    distance.append(res2[0]['properties']['distance'])
    if (i%50)>48:
        time.sleep(1)   # custom the stop time according to use policies. 
        print('=======%s======='%format(str(i)))
    print('------------------------')


check the results are homogenous. 

In [ ]:
print('rue     :', len(rue))
print('Numero  :', len(housenbr))
print('distance:', len(distance))
print('dataframe: ', data.shape())

append the recovered data to the dataframe 

In [ ]:
data['rue']=rue
data['numero']=housenbr
data['dist']=distance

 ## Export the data
 

In [ ]:
# name the resulting file
file='E:/Feina/Autonoms/CTG_Project/AOAngers3/travailW21/API_Angers_20190528.csv'

# export the data to the CSV file
data.to_csv(file, sep=';', encoding='utf-8')